In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import brent
import pandas as pd
import datetime
import h5py
from sklearn.metrics import accuracy_score

In [2]:
def get_data(h5_path, OuterFold, InnerFold):
    with h5py.File(h5_path, "r") as f:
        X_trn = f[f'outerfold_{OuterFold}/innerfold_{InnerFold}/trn/X'][:]
        y_trn = f[f'outerfold_{OuterFold}/innerfold_{InnerFold}/trn/y'][:]
        X_vld = f[f'outerfold_{OuterFold}/innerfold_{InnerFold}/vld/X'][:]
        y_vld = f[f'outerfold_{OuterFold}/innerfold_{InnerFold}/vld/y'][:]
        sid = f[f'outerfold_{OuterFold}/innerfold_{InnerFold}/vld/sid'][:]
    return X_trn,y_trn,X_vld,y_vld,sid

def REML(lamb, n, e_val_SHS, omega_sq):
    return (n-1)*np.log(sum(np.divide(omega_sq, (e_val_SHS+lamb)))) + sum(np.log(e_val_SHS+lamb))

def BLUP(y,Z=None,G=None):
    n = len(y)
    if Z is None:
        Z = np.eye(n)
    else:
        Z = np.asarray(Z)
    X = np.ones((n,1))
    
    S = np.eye(n)-1/n*np.ones((n,n))
    sqn = np.sqrt(n)
    
    #spectral decomposition
    if G is None:
        H = np.matmul(Z,Z.T) + sqn*np.eye(n)
    else:
        H = np.matmul(Z, np.matmul(G,Z.T)) + sqn*np.eye(n)
    
    e_val, V = np.linalg.eig(H)
    e_val = e_val - sqn
    SHS = np.matmul(S, np.matmul(H,S))
    e_val_SHS, V_SHS = np.linalg.eig(SHS)
    V_SHS = np.delete(V_SHS, -1, axis=1)
    e_val_SHS = np.delete(e_val_SHS, -1) - sqn
    
    omega = np.matmul(V_SHS.T,y)
    omega_sq = np.multiply(omega,omega)
    
    #minimize REML
    x0 = np.abs(np.min(e_val_SHS))+1e-09
    res = minimize(REML,x0,args=(n,e_val_SHS,omega_sq))                     
    lamb_opt = res.x
    
    #calc H_inv
    H_inv = np.matmul(V,np.divide(V,(e_val+lamb_opt)).T)
    
    #calc beta and u
    W = np.matmul(X.T, np.matmul(H_inv,X))
    beta = np.linalg.solve(W, np.matmul(X.T, np.matmul(H_inv,y)))
    if G is None:
        u = np.matmul(Z.T, np.matmul(H_inv, (y-np.matmul(X,beta))))
    else:
        u = np.matmul(G, np.matmul(Z.T, np.matmul(H_inv, (y-np.matmul(X,beta)))))
    
    return beta, u

def pred(beta,Z,u):
    return beta + np.matmul(Z, u)

def get_Z(X_tr,X_ts):
    X_tr = np.asarray(X_tr)
    X_ts = np.asarray(X_ts)
    n = len(X_tr)+len(X_ts)
    mafs = np.sum(X_tr, axis = 0)+np.sum(X_ts, axis = 0)
    mafs = (n - mafs)/(2*n)
    P = 2*(mafs-0.5)
    Z_tr = X_tr-P
    Z_ts = X_ts-P
    return Z_tr, Z_ts

def to_labels(probs, threshold):
    return (probs >= threshold).astype('int')

In [3]:
base = '../ML4RG/train_test_split/'
pt = 'atwell_LeafRoll_s'
h5_path = pt + '_strat.h5'


In [17]:
#Regression
OuterFold = 0
InnerFold = 'full'

#get train and test data
X_train, y_train, X_test, y_test, sid_test = get_data(h5_path, OuterFold, InnerFold)

#train model and get predictions
Z_train, Z_test = get_Z(X_train,X_test)
beta, u = BLUP(y=y_train, Z=Z_train)
y_pred = pred(beta,Z_test,u)


In [ ]:
#save predictions
data = pd.DataFrame({
    'ID': sid_test,
    'True': y_test,
    'Prediction': y_pred
})

data.to_csv(base + pt + '/outerFold' + str(OuterFold) + '/predictions_blup/Predictions_Test_' + str(OuterFold) + '_' + str(InnerFold) +'_BLUP_' + pt + datetime.datetime.now().strftime("%d-%b-%Y_%H-%M") + '.csv',
        sep=',', decimal='.', float_format='%.10f')

In [4]:
#Classification - get threshold
OuterFold = 0
thresholds_to_test = np.arange(0, 1, 0.001)
scores = []
for i in range(0,3):
    X_train, y_train, X_test, y_test, sid_test = get_data(h5_path, OuterFold, i)

    Z_train, Z_test = get_Z(X_train,X_test)
    beta, u = BLUP(y=y_train, Z=Z_train)
    y_prob = pred(beta,Z_test,u)
    
    scores.append([accuracy_score(y_test, to_labels(y_prob, t)) for t in thresholds_to_test])

scores = np.array(scores)
scores_mean = np.mean(scores, axis = 0)
threshold = np.argmax(scores, axis=1)*0.001
thr_mean = np.argmax(scores_mean)*0.001

In [5]:
#Classification - retrain on full dataset
OuterFold = 0
InnerFold = 'full'

#get train and test data
X_train, y_train, X_test, y_test, sid_test = get_data(h5_path, OuterFold, InnerFold)

#train model and get predictions
Z_train, Z_test = get_Z(X_train,X_test)
beta, u = BLUP(y=y_train, Z=Z_train)
y_prob = pred(beta,Z_test,u)

#get labels
y_pred = to_labels(y_prob, thr_mean)
accuracy_score(y_test, y_pred)

In [7]:
#save predictions
data = pd.DataFrame({
    'ID': sid_test,
    'True': y_test,
    'Prediction': y_pred
})

data.to_csv(base + pt + '/outerFold' + str(OuterFold) + '/predictions_blup/Predictions_Test_' + str(OuterFold) + '_' + str(InnerFold) +'_BLUP_treshold_'+ str(thr_mean) +'_'+ pt + datetime.datetime.now().strftime("%d-%b-%Y_%H-%M") + '.csv',
        sep=',', decimal='.', float_format='%.10f')